In [ ]:
pip install -U sentence-transformers

In [2]:
pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import os
import time
import jsonlines
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import numpy as np
from concurrent.futures import ThreadPoolExecutor

In [ ]:
# Load the model
model = SentenceTransformer('all-MPNet-base-v2')  # replace with your model

# Set the paths
# Set the paths
input_file = '/content/drive/My Drive/Colab Notebooks/Item1_wrds_html_text.jsonl'
output_file = '/content/drive/My Drive/Colab Notebooks/Item1_wrds_html_text_embeds_MPNET.jsonl'



In [ ]:
# Open output file
with jsonlines.open(output_file, mode='w') as writer:
    # Open input file
    with jsonlines.open(input_file, mode='r') as reader:

        batch = []
        for obj in reader:
            batch.append(obj)
            if len(batch) >= 1000:
                start_time = time.time()

                # Process batch
                for item in batch:
                    content = item['text']
                    
                    # Check if content is not empty
                    if content.strip():
                        # Break text into sentences
                        sentences = sent_tokenize(content)

                        # Count number of words in content
                        n_words = len(content.split())

                        # Embed each sentence and average embeddings
                        embeddings = model.encode(sentences)
                        avg_embedding = np.mean(embeddings, axis=0).tolist()

                        # Write data to file
                        writer.write({'file_id': item['file_id'], 'embedding': avg_embedding, 'n_words': n_words})
                    else:
                        # Write data for empty content
                        writer.write({'file_id': item['file_id'], 'embedding': [], 'n_words': 0})
                
                end_time = time.time()

                print(f"Processed batch of {len(batch)} files")
                print(f"Time taken: {end_time - start_time} seconds\n")
                
                # Clear the batch
                batch = []